In [ ]:
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from spswarehouse.credentials import google_config
import pandas as pd
from datetime import datetime, date


def build_service():

    credentials = ServiceAccountCredentials.from_json_keyfile_dict(
        google_config['service-account'],
        scopes=google_config['scopes']
    )
    
    drive_serv = build(
        'drive',
        'v3',
        credentials=credentials
    )

    sheet_serv = build(
        'sheets',
        'v4',
        credentials=credentials
    )
    
    
    return [drive_serv, sheet_serv]



def generate_copies(service, sheet_service):
    
    #This is an example of what the copied docs should look like.
    #"08.26.20 Atlas HS - GLT 10 Agenda"
    
    #Change this to be whatever the date of the GLT meeting is
    time_stamp = "09.16.20 "
    
    #These folder IDs are for testing
    #I've removed the names of folders, along with the folder IDs,
    #  for security reasons.
    folder_ids = {'TEST GLT 9': 'enter_folder_id_here',
                'TEST GLT 10': 'enter_folder_id_here',
                'TEST GLT 11': 'enter_folder_id_here',
                'TEST GLT 12': 'enter_folder_id_here'}
    
    
    
    base_document_id = 'enter_base_doc_id_here'

    copy_output = {}
    sheet_list = []
    
    checkSet = set()
    
    for glt_name, folder_id in folder_ids.items():
  
    #Can run this code to check for accidental duplicate folder ID's
    #         if folder_id not in checkSet:
    #             checkSet.add(folder_id)
    #             print("Good")
    #         else:
    #             print("Already in set:", folder_id)
        
        try:
            
            new_name = time_stamp + glt_name
            body = {
                'name': new_name
            }

            inner_list = []
            inner_list.append(glt_name)

            print("About to copy:", new_name, "from base doc")
            drive_response = service.files().copy(
            fileId=base_document_id, body=body).execute()

            file_id = drive_response['id']

            print("Created copy:", new_name, ".with id:", file_id)
            
            finalUrl = 'https://docs.google.com/document/d/' + file_id + '/'
            
            copy_output[glt_name] = finalUrl
            inner_list.append(finalUrl)

            file = service.files().get(fileId=file_id,
                                            fields='parents').execute()
            previous_parents = ",".join(file.get('parents'))    

            file = service.files().update(fileId=file_id,
                                        addParents=folder_id,
                                        removeParents=previous_parents,
                                        fields='id, parents').execute()
            print("\n")
            
            sheet_list.append(inner_list)
            
        except:
            pass
        
    df = pd.DataFrame.from_dict(copy_output, orient="index")
    
    
    now = datetime.now()
    today = date.today()
    current_time = now.strftime("%H_%M_%S")
    current_day = today.strftime("%m_%d_%Y_")
    csv_name = current_day + current_time + ".csv"
    
    df.to_csv(csv_name)

    tab_name = time_stamp.strip()
    
    body = {
    'requests': [{
        'addSheet': {
            'properties': {
                'title': tab_name,
            }
        }
        }]
    }
    
    #Change the ID of this to where the GLT Agendas spreadsheet is
    glt_sheet_id = 'enter_sheet_id_here'
    try:
        result = sheet_service.spreadsheets().batchUpdate(
        spreadsheetId=glt_sheet_id,
        body=body).execute()

        resp = sheet_service.spreadsheets().values().append(
               spreadsheetId=glt_sheet_id,
               valueInputOption='RAW',
               range="\'" + tab_name  + "\'" + "!A2",
               body=dict(
                   majorDimension='ROWS',
                   values=sheet_list
               )
           ).execute()

    except:
        pass
    
    return copy_output

In [ ]:
if __name__ == '__main__':
    google_serv = build_service()
    copy_out = generate_copies(google_serv[0], google_serv[1])

In [ ]:

#Visual check of copies created

for k, v in copy_out.items():
    print("Doc Name:", k, " | ", "Doc ID:", v)